In [57]:
import cv2
import numpy as np
import math
import statistics

In [58]:
def read_fingerprints(filename):
    '''
        Taken directly from author: Ahron Wayne.
        Reads a space-separated file containing x, y, z coordinates
        directly into an np array of the same shape.
        The z-coordinate represents depth.
    '''
    with open(filename) as f:
        fingerprints = f.readlines()
    fingerprints = [x.strip() for x in fingerprints]
    fingerprints = [x.split() for x in fingerprints]
    fingerprints = np.array(fingerprints, dtype=float)
    return fingerprints

In [59]:
fingerprints_np = read_fingerprints('fingerprintsdata3.txt')
fingerprints_np.shape

(134894, 3)

In [60]:
from alibi_detect.od import Mahalanobis

from alibi_detect.base import BaseDetector

In [61]:
def detect_outlier_mask(arr: np.array, outlier_model: BaseDetector) -> np.array:
    '''
        Takes (-1,3) dimension array and an outlier detector model
        which returns an np array of dimension (-1,), a mapping
        of 0, 1 values corresponding to no outlier, outlier.
    '''
    assert len(arr.shape) == 2 and arr.shape[1] == 3
    z_only_arr = np.expand_dims(arr[:,2], axis=1)
    return outlier_model.predict(z_only_arr)['data']['is_outlier']

In [65]:
outlier_mask_np = detect_outlier_mask(fingerprints_np, Mahalanobis(threshold=1))
print(f"Percent of outliers is { np.mean(outlier_mask_np) }")

Percent of outliers is 0.661178406749003


In [63]:
fingerprints_no_outliers_np = np.copy(fingerprints_np)
delete_indices = np.where(outlier_mask_np == 1)
fingerprints_no_outliers_np = np.delete(fingerprints_no_outliers_np, delete_indices, 0)

In [64]:
np.savetxt('removed_outliers_fingerprintsdata3.txt', fingerprints_no_outliers_np, fmt="%f6", delimiter=" ")